In [1]:
#@title ✅ Environment setup
import os, time, json, random, requests
from typing import Dict, Any

# these will point to your running APIs
PROJECT_API = os.environ.get("PROJECT_API", "http://127.0.0.1:8000")
SD_API      = os.environ.get("SD_API", "http://127.0.0.1:8001")

print("🔗 Connected Endpoints:")
print("PROJECT_API:", PROJECT_API)
print("SD_API:", SD_API)


🔗 Connected Endpoints:
PROJECT_API: http://127.0.0.1:8000
SD_API: http://127.0.0.1:8001


In [2]:
#@title ✅ Tool stubs for project model, Stable Diffusion, and CSV summarizer
def tool_project_qa(query: str) -> Dict[str, Any]:
    """Call your Week 7 backend (Graph-RAG / Multi-Hop)."""
    try:
        r = requests.post(f"{PROJECT_API}/qa", json={"text": query}, timeout=30)
        return r.json()
    except Exception as e:
        return {"error": str(e)}

def tool_sd_generate(prompt: str) -> Dict[str, Any]:
    """Call Stable Diffusion backend that exposes /generate."""
    try:
        r = requests.post(f"{SD_API}/generate", json={"prompt": prompt}, timeout=60)
        return r.json()
    except Exception as e:
        return {"error": str(e)}

def tool_csv_summary(csv_path: str) -> Dict[str, Any]:
    """Tiny utility tool — summarize CSV numerically."""
    import pandas as pd
    try:
        df = pd.read_csv(csv_path)
        desc = df.describe(include="all").to_dict()
        return {"summary": desc}
    except Exception as e:
        return {"error": str(e)}


In [3]:
#@title ✅ Planner, guardrails & agent logic
BLOCKLIST = ["malware", "ssn", "credit card", "phi"]

def guardrails(q: str) -> bool:
    return not any(b in q.lower() for b in BLOCKLIST)

def planner(query: str) -> str:
    """Route query to correct tool."""
    if "image" in query.lower():
        return "sd"
    if "csv" in query.lower() or "data" in query.lower():
        return "csv"
    return "qa"

def run_agent(query: str) -> Dict[str, Any]:
    if not guardrails(query):
        return {"answer": "🚫 Refused by guardrails.", "trace": [{"step": "blocked"}]}

    route = planner(query)
    trace = [{"plan": route}]
    print(f"🤖 Planner chose route → {route.upper()}")

    if route == "qa":
        out = tool_project_qa(query)
        trace.append({"qa_result": out})
        return {
            "answer": out.get("answer", "(no answer)"),
            "citations": out.get("citations", []),
            "trace": trace
        }

    elif route == "sd":
        out = tool_sd_generate(query)
        trace.append({"sd_result": out})
        return {
            "answer": "🖼️ Image generated successfully.",
            "assets": out,
            "trace": trace
        }

    elif route == "csv":
        csv_path = "evaluation_log.csv"
        out = tool_csv_summary(csv_path)
        trace.append({"csv_result": out})
        return {
            "answer": "📊 CSV Summary ready.",
            "summary": out,
            "trace": trace
        }


In [4]:
#@title ✅ Run demo queries (Agent reasoning trace)
demo_queries = [
    "Summarize AI market performance from the project model",
    "Generate an image for FinSynth poster banner (image, teal theme)",
    "Give me quick statistics of the evaluation_log.csv"
]

for q in demo_queries:
    print("🔹 QUERY:", q)
    out = run_agent(q)
    print(json.dumps(out, indent=2)[:600])
    print("─" * 80)


🔹 QUERY: Summarize AI market performance from the project model
🤖 Planner chose route → QA
{
  "answer": "(no answer)",
  "citations": [],
  "trace": [
    {
      "plan": "qa"
    },
    {
      "qa_result": {
        "error": "HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /qa (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b01f0c04770>: Failed to establish a new connection: [Errno 111] Connection refused'))"
      }
    }
  ]
}
────────────────────────────────────────────────────────────────────────────────
🔹 QUERY: Generate an image for FinSynth poster banner (image, teal theme)
🤖 Planner chose route → SD
{
  "answer": "\ud83d\uddbc\ufe0f Image generated successfully.",
  "assets": {
    "error": "HTTPConnectionPool(host='127.0.0.1', port=8001): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b01f0c36b70>: Failed to establish a new connection: [Err

In [5]:
#@title ✅ Agent FastAPI endpoint (agent_api.py)
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="FinSynth Agent API")

class Query(BaseModel):
    text: str

@app.post("/agent")
def agent_endpoint(q: Query):
    res = run_agent(q.text)
    return res


In [6]:
#@title ✅ Save trace logs
import pandas as pd
traces = []
for q in demo_queries:
    res = run_agent(q)
    traces.append({
        "query": q,
        "route": res["trace"][0]["plan"],
        "timestamp": time.time()
    })
trace_df = pd.DataFrame(traces)
trace_df.to_csv("agent_trace_log.csv", index=False)
trace_df


🤖 Planner chose route → QA
🤖 Planner chose route → SD
🤖 Planner chose route → CSV


,query,route,timestamp
0,Summarize AI market performance from the proje...,qa,1.760409e+09
1,Generate an image for FinSynth poster banner (...,sd,1.760409e+09
2,Give me quick statistics of the evaluation_log...,csv,1.760409e+09
